In [1]:
print("fusion part")

fusion part


In [2]:
#vocal library import
import librosa
import numpy as np
import pandas as pd
from pydub import AudioSegment, effects
import os

import speech_recognition as src
from pydub.silence import split_on_silence
import wavio as wv
from scipy.io.wavfile import write, read
import sounddevice as sd
import matplotlib.pyplot as plt
from nltk.sentiment import SentimentIntensityAnalyzer

/Users/usama/anaconda3/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
# face library import
import tensorflow as tf 
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2 
import time

importing models

In [4]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
model = tf.keras.models.load_model('saved_training_50epo.h5')
model_v = tf.keras.models.load_model('my_model_weights_100epo.h5')


processing the voice to text

In [5]:
test_path = 'test.wav'
dff = pd.DataFrame([test_path], columns=['path']),
df = pd.concat(dff, axis=1)
df

,path
0,test.wav


In [6]:
def preprocess_audio(path):
    _, sr = librosa.load(path)
 
    raw_audio = AudioSegment.from_file(path)
    
    samples = np.array(raw_audio.get_array_of_samples(), dtype='float32')
    trimmed, _ = librosa.effects.trim(samples, top_db=25)
    padded = np.pad(trimmed, (0, 180000-len(trimmed)), 'constant')
    return padded, sr

In [7]:
r = src.Recognizer()
def transcribe_audio(path):
    # use the audio file as the audio source
    with src.AudioFile(path) as source:
        audio_listened = r.record(source)
        # try converting it to text
        text = r.recognize_google(audio_listened)
    return text

In [8]:

def get_large_audio_transcription_on_silence(path):
    """Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks"""
    # open the audio file using pydub
    sound = AudioSegment.from_file(path)  
    # split audio sound where silence is 500 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        try:
            text = transcribe_audio(chunk_filename)
        except src.UnknownValueError as e:
            print("Error:", str(e))
        else:
            text = f"{text.capitalize()}. "
            print(chunk_filename, ":", text)
            whole_text += text
    # return the text for all chunks detected
    return whole_text

In [9]:
import threading
# import time
# timeout = time.time() + 18  # 18 seconds from now

In [10]:
def face_func():
    print("Thread Vision started")
    test_img = cv2.imread('cry.jpg')
    gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_img, 1.1,4)
    for x,y,z,h in faces:
        r_gray = gray_img[y:y+h, x:x+z]
        r_colour = test_img[y:y+h, x:x+z]
        cv2.rectangle(test_img, (x,y), (x+z, y+h), (0,255,0), 2)
        faces1 = face_cascade.detectMultiScale(r_gray)
        if len(faces1) == 0:
            print("no face detected")
        else:
            for a,b,c,d in faces1:
                face_r = r_colour[b:b+d, a:a+c]
    #plt.imshow(cv2.cvtColor(face_r, cv2.COLOR_BGR2BGRA))
    final_img = cv2.resize(face_r, (224,224))
    final_img = np.expand_dims(final_img,axis=0)
    final_img = final_img/255
    Predictions = model.predict(final_img)
    Predictions = np.argmax(Predictions)
   
    font_scale=1.5
    font = cv2.FONT_HERSHEY_PLAIN
    # set the rectangle background to white
    rectangle_bgr = (255, 255, 255)
    # make a black image
    img = np.zeros((500, 500))
    # set some text
    text = "Some text in a box!"
    # get the width and height of the text box
    (text_width, text_height) = cv2.getTextSize(text, font, fontScale=font_scale, thickness=1)[0] # set the text start position
    text_offset_x = 10
    text_offset_y= img.shape[0] - 25
    #make the coords of the box with a small padding of two pixels
    box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width+2, text_offset_y - text_height - 2)) 
    cv2.rectangle(img, box_coords[0], box_coords[1], rectangle_bgr, cv2.FILLED)
    cv2.putText(img, text, (text_offset_x, text_offset_y), font, fontScale=font_scale, color=(0, 0, 0), thickness=1)

    
    cap = cv2.VideoCapture(0)
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Cannot open webcam")


    data = []

    for i in range(80):
        # if time.time() > timeout:
        #     break
        ret, frame = cap.read()
        #eye_cascade = cv2.Cascade Classifier (cv2.data. haarcascades +'haarcascade_eye.xml')
        faceCascade = cv2.CascadeClassifier (cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') 
        gray_img = cv2.cvtColor (frame, cv2.COLOR_BGR2GRAY)
        #print (faceCascade.empty())
        faces = face_cascade.detectMultiScale(gray_img, 1.1,4)
        for x,y,z,h in faces:
            r_gray = gray_img[y:y+h, x:x+z]
            r_colour = frame[y:y+h, x:x+z]
            cv2.rectangle(test_img, (x,y), (x+z, y+h), (0,255,0), 2)
            faces1 = face_cascade.detectMultiScale(r_gray)
            if len(faces1) == 0:
                print("no face detected")
            else:
                for a,b,c,d in faces1:
                    face_r = r_colour[b:b+d, a:a+c]

        ## cropping the face
        final_img = cv2.resize(face_r, (224,224))
        final_img = np.expand_dims(final_img,axis=0)
        final_img = final_img/255

        font = cv2.FONT_HERSHEY_SIMPLEX
        Predictions = model.predict(final_img)

        #save to csv filee
        data.append(np.argmax(Predictions))
        df_face = pd.DataFrame(data, columns=['Face'])
        
        df_face.to_csv('face_outputs.csv', index=False)


        # pd.DataFrame(Predictions, columns=['face']).to_csv('prediction.csv')
        font_scale = 1.5
        font = cv2.FONT_HERSHEY_PLAIN
        if (np.argmax (Predictions)==0): 
            status = "Angry"
            x1,y1, w1,h1 = 0,0,175,75
            # Draw black background rectangle
            cv2.rectangle(frame, (x1, x1), (x1+ w1, y1 + h1), (0,0,0), -1)
            # Add text
            cv2.putText(frame, status, (x1+ int(w1/10), y1 + int (h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255), 2, cv2.LINE_4)
            cv2.rectangle (frame, (x, y), (x+z, y+h), (0, 0, 255))

        elif (np.argmax (Predictions)==1):
            status = "Disgust"
            x1,y1,w1,h1 = 0,0,175,75
            # Draw black background rectangle
            cv2.rectangle(frame, (x1, x1), (x1+ w1, y1 + h1), (0,0,0), -1)
            # Add text
            cv2.putText(frame, status, (x1+ int(w1/10), y1 + int (h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255),2,cv2.LINE_4)
            cv2.rectangle(frame, (x, y), (x+z, y+h), (0, 0, 255))

        elif (np.argmax (Predictions)==2):
            status = "Fear"
            x1,y1,w1,h1 = 0,0,175,75
            # Draw black background rectangle
            cv2.rectangle(frame, (x1, x1), (x1+ w1, y1 + h1), (0,0,0), -1)
            # Add text
            cv2.putText(frame, status, (x1+ int(w1/10), y1 + int (h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255),2,cv2.LINE_4)
            cv2.rectangle(frame, (x, y), (x+z, y+h), (0, 0, 255))

        elif (np.argmax (Predictions)==3):
            status = "Happy"
            x1,y1,w1,h1 = 0,0,175,75
            # Draw black background rectangle
            cv2.rectangle(frame, (x1, x1), (x1+ w1, y1 + h1), (0,0,0), -1)
            # Add text
            cv2.putText(frame, status, (x1+ int(w1/10), y1 + int (h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255),2,cv2.LINE_4)
            cv2.rectangle(frame, (x, y), (x+z, y+h), (0, 0, 255))

        elif (np.argmax (Predictions)==5):
            status = "Sad"
            x1,y1,w1,h1 = 0,0,175,75
            # Draw black background rectangle
            cv2.rectangle(frame, (x1, x1), (x1+ w1, y1 + h1), (0,0,0), -1)
            # Add text
            cv2.putText(frame, status, (x1+ int(w1/10), y1 + int (h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255),2,cv2.LINE_4)
            cv2.rectangle(frame, (x, y), (x+z, y+h), (0, 0, 255))
        
        elif (np.argmax (Predictions)==6):
            status = "Surprised"
            x1,y1,w1,h1 = 0,0,175,75
            # Draw black background rectangle
            cv2.rectangle(frame, (x1, x1), (x1+ w1, y1 + h1), (0,0,0), -1)
            # Add text
            cv2.putText(frame, status, (x1+ int(w1/10), y1 + int (h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255),2,cv2.LINE_4)
            cv2.rectangle(frame, (x, y), (x+z, y+h), (0, 0, 255))

        else:
            status = "Neutral"
            x1,y1,w1,h1 = 0,0,175,75
            # Draw black background rectangle
            cv2.rectangle(frame, (x1, x1), (x1+ w1, y1 + h1), (0,0,0), -1)
            # Add text
            cv2.putText(frame, status, (x1+ int(w1/10), y1 + int (h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255),2,cv2.LINE_4)
            cv2.rectangle(frame, (x, y), (x+z, y+h), (0, 0, 255))


        cv2.imshow('face emotions',frame)
        if cv2.waitKey(3) & 0xFF == ord('q'):
            cv2.destroyWindow('face emotions')
            break


In [11]:
def thread_two_func():
    print("Thread voice and text started")
    
    print('say hi')

    data_voice = []
    
    for i in range(4):
        # if time.time() > timeout:
        #     break

        #init the fequancy and the time of the .wav file (duration of listening)
        freq = 44100
        duration = 4
        zcr_list = []
        rms_list = []
        mfccs_list = []

        #record a .wav file
        recording = sd.rec(int(duration * freq), samplerate=freq, channels=1)
        sd.wait()
        write("recording0.wav", freq, recording)
        wv.write("recording1.wav", recording, freq, sampwidth=2)
        

        test_path = 'recording1.wav'
        
        #converting the data from vocal to textual
        text =  get_large_audio_transcription_on_silence(test_path)
        print(text)

        #get the senitmet from the text
        sia = SentimentIntensityAnalyzer()
        my_dict = sia.polarity_scores(text)

        #get the value and the intensity of the text
        max_value = max(my_dict.values())
        max_key = max(my_dict, key=my_dict.get)
        
        dff = pd.DataFrame([test_path], columns=['path']),
        df = pd.concat(dff, axis=1)
        
        #init the frame of the vocal 
        FRAME_LENGTH = 2048
        HOP_LENGTH = 512

        #iterrate through our .wav file that we recored
        for index, row in df.iterrows():
            try:
                y, sr = preprocess_audio(row.path)
                zcr = librosa.feature.zero_crossing_rate(y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
                rms = librosa.feature.rms(y=y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
                mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=HOP_LENGTH)

                zcr_list.append(zcr)
                rms_list.append(rms)
                mfccs_list.append(mfccs)
                 
            except:
                print(f"Failed for path: {row.path}")
        
        #preparing the input to the model
        try:
            X_test = np.concatenate((
                np.swapaxes(zcr_list, 1, 2), 
                np.swapaxes(rms_list, 1, 2), 
                np.swapaxes(mfccs_list, 1, 2)), 
                axis=2
            )
            
            X_test = X_test.astype('float32')
            y_pred = np.argmax(model_v.predict(X_test), axis=1)
            
            #save to csv
            data_voice.append([ y_pred,max_value, text,  max_key])
            df_voice = pd.DataFrame(data_voice, columns=[ 'voice','text_intensity', 'text', 'text_sentiment' ])
            
            df_voice.to_csv('voice_outputs.csv', index=False)

            

        except:
            print("no data have been passed")
        
        emotion_dic = {'neutral' : 0, 'happy'   : 1,'sad'     : 2, 'angry'   : 3, 'fear'    : 4, 'disgust' : 5}
    
        print(list(emotion_dic.keys())[list(emotion_dic.values()).index(int(y_pred))])
        

In [12]:
# Create thread for alignment
thread_two = threading.Thread(target=thread_two_func)

# Start the thread
thread_two.start()

# Wait for threads to complete while continuing the parant 
face_func()
thread_two.join()

print("Thread execution complete")



Thread voice and text started
say hi
Thread Vision started
1/1 [==============================] - 0s 27ms/step
no face detected
1/1 [==============================] - 0s 29ms/step


/Users/usama/anaconda3/lib/python3.10/site-packages/wavio.py:259: ClippedDataWarning: Some data values have been clipped.  With scale=1.0, the interval of input values that will not be clipped is [-1.0000305180437934, 1.0]
  _warnings.warn(ClippedDataWarning(msg))


1/1 [==============================] - 0s 28ms/step
audio-chunks/chunk1.wav : Stockport to my existence file. 
Stockport to my existence file. 
1/1 [==============================] - 0s 52ms/step
no face detected
1/1 [==============================] - 2s 2s/step
angry
1/1 [==============================] - 0s 66ms/step
audio-chunks/chunk1.wav : Sorry it's a bit slow since at the moment. 
Sorry it's a bit slow since at the moment. 
1/1 [==============================] - 0s 233ms/step
fear
1/1 [==============================] - 0s 53ms/step
no face detected
1/1 [==============================] - 0s 54ms/step


/Users/usama/anaconda3/lib/python3.10/site-packages/wavio.py:259: ClippedDataWarning: Some data values have been clipped.  With scale=1.0, the interval of input values that will not be clipped is [-1.0000305180437934, 1.0]
  _warnings.warn(ClippedDataWarning(msg))


no face detected
1/1 [==============================] - 0s 103ms/step
Error: 
1/1 [==============================] - 0s 63ms/step
audio-chunks/chunk2.wav : Spotify. 
Spotify. 
1/1 [==============================] - 0s 71ms/step
angry
1/1 [==============================] - 0s 71ms/step
audio-chunks/chunk1.wav : It will automatically run through. 
It will automatically run through. 
1/1 [==============================] - 0s 78ms/step
fear
Thread execution complete


In [13]:
# process the csv files to have clean output

v_df = pd.read_csv("voice_outputs.csv")
for index in v_df.index:
    if v_df['text_intensity'][index] == 0.000:
        v_df['text_sentiment'][index] = 'neu'

v_df['voice'] = v_df['voice'].str[1]  
v_df['voice'] = v_df['voice'].apply(lambda arr: arr[0]).apply(pd.to_numeric, downcast='integer')
v_df.loc[v_df['text'].isnull(),'voice'] = 0


#process and aligine face with voice data
f_df = pd.read_csv('face_outputs.csv')
num_groups = len(f_df) // 20
grouped_df = [f_df.iloc[i * 4:(i + 1) * 4] for i in range(num_groups)]
averages = [group['Face'].mode().iloc[0] for group in grouped_df]
face_df = pd.DataFrame({'face_senti': averages})

#concat all outputs in one dataframe
final_df = pd.concat([face_df, v_df], axis=1)
final_df.to_csv('final_collective_sentiments.csv', index=False)
final_df

,face_senti,voice,text_intensity,text,text_sentiment
0,4,3,1.000,Stockport to my existence file.,neu
1,3,4,0.843,Sorry it's a bit slow since at the moment.,neu
2,4,3,1.000,Spotify.,neu
3,3,4,1.000,It will automatically run through.,neu


In [14]:
def predict_emotion(face_emotion, voice_emotion, text_sentiment):
    emotion_mapping = {
        # Single Emotions
        ('Happy', 'Happy', 'Positive'): 'Happy',
        ('Sad', 'Sad', 'Negative'): 'Sad',
        ('Angry', 'Angry', 'Negative'): 'Angry',
        ('Fearful', 'Fearful', 'Negative'): 'Fearful',
        ('Disgusted', 'Disgusted', 'Negative'): 'Disgusted',
        ('Neutral', 'Neutral', 'Neutral'): 'Neutral',

        # Mixed Emotions with happy face
        ('Happy', 'Neutral', 'Positive'): 'Mixed Happy-Neutral-Positive',
        ('Happy', 'Sad', 'Positive'): 'Mixed Happy-Sad-Positive',
        ('Happy', 'Angry', 'Positive'): 'Mixed Happy-Angry-Positive',
        ('Happy', 'Fearful', 'Positive'): 'Mixed Happy-Fearful-Positive',
        ('Happy', 'Disgusted', 'Positive'): 'Mixed Happy-Disgusted-Positive',
        ('Happy', 'Neutral', 'Neutral'): 'Mixed Happy-Neutral-Neutral',
        ('Happy', 'Sad', 'Neutral'): 'Mixed Happy-Sad-Neutral',
        ('Happy', 'Angry', 'Neutral'): 'Mixed Happy-Angry-Neutral',
        ('Happy', 'Fearful', 'Neutral'): 'Mixed Happy-Fearful-Neutral',
        ('Happy', 'Happy', 'Neutral'): 'Mixed Happy-Happy-Neutral',
        ('Happy', 'Disgusted', 'Neutral'): 'Mixed Happy-Disgusted-Neutral',
        ('Happy', 'Neutral', 'Negative'): 'Mixed Happy-Neutral-Negative',
        ('Happy', 'Sad', 'Negative'): 'Mixed Happy-Sad-Negative',
        ('Happy', 'Angry', 'Negative'): 'Mixed Happy-Angry-Negative',
        ('Happy', 'Fearful', 'Negative'): 'Mixed Happy-Fearful-Negative',
        ('Happy', 'Happy', 'Negative'): 'Mixed Happy-Happy-Negative',
        ('Happy', 'Disgusted', 'Negative'): 'Mixed Happy-Disgusted-Negative',

         # Mixed Emotions with Angry face
        ('Angry', 'Neutral', 'Positive'): 'Mixed Angry-Neutral-Positive',
        ('Angry', 'Sad', 'Positive'): 'Mixed Angry-Sad-Positive',
        ('Angry', 'Angry', 'Positive'): 'Mixed Angry-Angry-Positive',
        ('Angry', 'Fearful', 'Positive'): 'Mixed Angry-Fearful-Positive',
        ('Angry', 'Happy', 'Positive'): 'Mixed Angry-Happy-Positive',
        ('Angry', 'Disgusted', 'Positive'): 'Mixed Angry-Disgusted-Positive',
        ('Angry', 'Neutral', 'Neutral'): 'Mixed Angry-Neutral-Neutral',
        ('Angry', 'Sad', 'Neutral'): 'Mixed Angry-Sad-Neutral',
        ('Angry', 'Angry', 'Neutral'): 'Mixed Angry-Angry-Neutral',
        ('Angry', 'Fearful', 'Neutral'): 'Mixed Angry-Fearful-Neutral',
        ('Angry', 'Happy', 'Neutral'): 'Mixed Angry-Happy-Neutral',
        ('Angry', 'Disgusted', 'Neutral'): 'Mixed Angry-Disgusted-Neutral',
        ('Angry', 'Neutral', 'Negative'): 'Mixed Angry-Neutral-Negative',
        ('Angry', 'Sad', 'Negative'): 'Mixed Angry-Sad-Negative',
        ('Angry', 'Fearful', 'Negative'): 'Mixed Angry-Fearful-Negative',
        ('Angry', 'Happy', 'Negative'): 'Mixed Angry-Happy-Negative',
        ('Angry', 'Disgusted', 'Negative'): 'Mixed Angry-Disgusted-Negative',

       # Mixed Emotions with Disgust face
        ('Disgust', 'Neutral', 'Positive'): 'Mixed Disgust-Neutral-Positive',
        ('Disgust', 'Sad', 'Positive'): 'Mixed Disgust-Sad-Positive',
        ('Disgust', 'Angry', 'Positive'): 'Mixed Disgust-Angry-Positive',
        ('Disgust', 'Fearful', 'Positive'): 'Mixed Disgust-Fearful-Positive',
        ('Disgust', 'Happy', 'Positive'): 'Mixed Disgust-Happy-Positive',
        ('Disgust', 'Disgusted', 'Positive'): 'Mixed Disgust-Disgusted-Positive',
        ('Disgust', 'Neutral', 'Neutral'): 'Mixed Disgust-Neutral-Neutral',
        ('Disgust', 'Sad', 'Neutral'): 'Mixed Disgust-Sad-Neutral',
        ('Disgust', 'Angry', 'Neutral'): 'Mixed Disgust-Angry-Neutral',
        ('Disgust', 'Fearful', 'Neutral'): 'Mixed Disgust-Fearful-Neutral',
        ('Disgust', 'Happy', 'Neutral'): 'Mixed Disgust-Happy-Neutral',
        ('Disgust', 'Disgusted', 'Neutral'): 'Mixed Disgust-Disgusted-Neutral',
        ('Disgust', 'Neutral', 'Negative'): 'Mixed Disgust-Neutral-Negative',
        ('Disgust', 'Sad', 'Negative'): 'Mixed Disgust-Sad-Negative',
        ('Disgust', 'Angry', 'Negative'): 'Mixed Disgust-Angry-Negative',
        ('Disgust', 'Fearful', 'Negative'): 'Mixed Disgust-Fearful-Negative',
        ('Disgust', 'Happy', 'Negative'): 'Mixed Disgust-Happy-Negative',
        
        # Mixed Emotions with Fear face
        ('Fear', 'Neutral', 'Positive'): 'Mixed Fear-Neutral-Positive',
        ('Fear', 'Sad', 'Positive'): 'Mixed Fear-Sad-Positive',
        ('Fear', 'Angry', 'Positive'): 'Mixed Fear-Angry-Positive',
        ('Fear', 'Fearful', 'Positive'): 'Mixed Fear-Fearful-Positive',
        ('Fear', 'Happy', 'Positive'): 'Mixed Fear-Happy-Positive',
        ('Fear', 'Disgusted', 'Positive'): 'Mixed Fear-Disgusted-Positive',
        ('Fear', 'Neutral', 'Neutral'): 'Mixed Fear-Neutral-Neutral',
        ('Fear', 'Sad', 'Neutral'): 'Mixed Fear-Sad-Neutral',
        ('Fear', 'Angry', 'Neutral'): 'Mixed Fear-Angry-Neutral',
        ('Fear', 'Fearful', 'Neutral'): 'Mixed Fear-Fearful-Neutral',
        ('Fear', 'Happy', 'Neutral'): 'Mixed Fear-Happy-Neutral',
        ('Fear', 'Disgusted', 'Neutral'): 'Mixed Fear-Disgusted-Neutral',
        ('Fear', 'Neutral', 'Negative'): 'Mixed Fear-Neutral-Negative',
        ('Fear', 'Sad', 'Negative'): 'Mixed Fear-Sad-Negative',
        ('Fear', 'Angry', 'Negative'): 'Mixed Fear-Angry-Negative',
        ('Fear', 'Happy', 'Negative'): 'Mixed Fear-Happy-Negative',
        ('Fear', 'Disgusted', 'Negative'): 'Mixed Fear-Disgusted-Negative',

        # Mixed Emotions with Neutral face
        ('Neutral', 'Neutral', 'Positive'): 'Mixed Neutral-Neutral-Positive',
        ('Neutral', 'Sad', 'Positive'): 'Mixed Neutral-Sad-Positive',
        ('Neutral', 'Angry', 'Positive'): 'Mixed Neutral-Angry-Positive',
        ('Neutral', 'Fearful', 'Positive'): 'Mixed Neutral-Fearful-Positive',
        ('Neutral', 'Happy', 'Positive'): 'Mixed Neutral-Happy-Positive',
        ('Neutral', 'Disgusted', 'Positive'): 'Mixed Neutral-Disgusted-Positive',
        ('Neutral', 'Sad', 'Neutral'): 'Mixed Neutral-Sad-Neutral',
        ('Neutral', 'Angry', 'Neutral'): 'Mixed Neutral-Angry-Neutral',
        ('Neutral', 'Fearful', 'Neutral'): 'Mixed Neutral-Fearful-Neutral',
        ('Neutral', 'Happy', 'Neutral'): 'Mixed Neutral-Happy-Neutral',
        ('Neutral', 'Disgusted', 'Neutral'): 'Mixed Neutral-Disgusted-Neutral',
        ('Neutral', 'Neutral', 'Negative'): 'Mixed Neutral-Neutral-Negative',
        ('Neutral', 'Sad', 'Negative'): 'Mixed Neutral-Sad-Negative',
        ('Neutral', 'Angry', 'Negative'): 'Mixed Neutral-Angry-Negative',
        ('Neutral', 'Fearful', 'Negative'): 'Mixed Neutral-Fearful-Negative',
        ('Neutral', 'Happy', 'Negative'): 'Mixed Neutral-Happy-Negative',
        ('Neutral', 'Disgusted', 'Negative'): 'Mixed Neutral-Disgusted-Negative',

        # Mixed Emotions with Sad face
        ('Sad', 'Neutral', 'Positive'): 'Mixed Sad-Neutral-Positive',
        ('Sad', 'Sad', 'Positive'): 'Mixed Sad-Sad-Positive',
        ('Sad', 'Angry', 'Positive'): 'Mixed Sad-Angry-Positive',
        ('Sad', 'Fearful', 'Positive'): 'Mixed Sad-Fearful-Positive',
        ('Sad', 'Happy', 'Positive'): 'Mixed Sad-Happy-Positive',
        ('Sad', 'Disgusted', 'Positive'): 'Mixed Sad-Disgusted-Positive',
        ('Sad', 'Neutral', 'Neutral'): 'Mixed Sad-Neutral-Neutral',
        ('Sad', 'Sad', 'Neutral'): 'Mixed Sad-Sad-Neutral',
        ('Sad', 'Angry', 'Neutral'): 'Mixed Sad-Angry-Neutral',
        ('Sad', 'Fearful', 'Neutral'): 'Mixed Sad-Fearful-Neutral',
        ('Sad', 'Happy', 'Neutral'): 'Mixed Sad-Happy-Neutral',
        ('Sad', 'Disgusted', 'Neutral'): 'Mixed Sad-Disgusted-Neutral',
        ('Sad', 'Neutral', 'Negative'): 'Mixed Sad-Neutral-Negative',
        ('Sad', 'Angry', 'Negative'): 'Mixed Sad-Angry-Negative',
        ('Sad', 'Fearful', 'Negative'): 'Mixed Sad-Fearful-Negative',
        ('Sad', 'Happy', 'Negative'): 'Mixed Sad-Happy-Negative',
        ('Sad', 'Disgusted', 'Negative'): 'Mixed Sad-Disgusted-Negative',

        # Mixed Emotions with Surprise face
        ('Surprise', 'Neutral', 'Positive'): 'Mixed Surprise-Neutral-Positive',
        ('Surprise', 'Sad', 'Positive'): 'Mixed Surprise-Sad-Positive',
        ('Surprise', 'Angry', 'Positive'): 'Mixed Surprise-Angry-Positive',
        ('Surprise', 'Fearful', 'Positive'): 'Mixed Surprise-Fearful-Positive',
        ('Surprise', 'Happy', 'Positive'): 'Mixed Surprise-Happy-Positive',
        ('Surprise', 'Disgusted', 'Positive'): 'Mixed Surprise-Disgusted-Positive',
        ('Surprise', 'Neutral', 'Neutral'): 'Mixed Surprise-Neutral-Neutral',
        ('Surprise', 'Sad', 'Neutral'): 'Mixed Surprise-Sad-Neutral',
        ('Surprise', 'Angry', 'Neutral'): 'Mixed Surprise-Angry-Neutral',
        ('Surprise', 'Fearful', 'Neutral'): 'Mixed Surprise-Fearful-Neutral',
        ('Surprise', 'Happy', 'Neutral'): 'Mixed Surprise-Happy-Neutral',
        ('Surprise', 'Disgusted', 'Neutral'): 'Mixed Surprise-Disgusted-Neutral',
        ('Surprise', 'Neutral', 'Negative'): 'Mixed Surprise-Neutral-Negative',
        ('Surprise', 'Sad', 'Negative'): 'Mixed Surprise-Sad-Negative',
        ('Surprise', 'Angry', 'Negative'): 'Mixed Surprise-Angry-Negative',
        ('Surprise', 'Fearful', 'Negative'): 'Mixed Surprise-Fearful-Negative',
        ('Surprise', 'Happy', 'Negative'): 'Mixed Surprise-Happy-Negative',
        ('Surprise', 'Disgusted', 'Negative'): 'Mixed Surprise-Disgusted-Negative',

        
        # Add other combinations for Neutral-Neutral-X and X-Neutral-Neutral if needed

    }

    # Check if the input combination exists in the mapping
    key = (face_emotion, voice_emotion, text_sentiment)
    if key in emotion_mapping:
        return emotion_mapping[key]
    else:
        return 'Unknown'

face_numeric_to_emotion = {
    0: 'Angry',
    1: 'Disgust',
    2: 'Fear',
    3: 'Happy',
    4: 'Neutral',
    5: 'Sad',
    6: 'Surprise'
}

voice_numeric_to_emotion = {
    0 : 'Neutral',
    1 : 'Happy' ,
    2 : 'Sad', 
    3 : 'Angry', 
    4 : 'Fearful'   , 
    5 : 'Disgusted'
}

text_mapping = {
    'neu' : 'Neutral',
    'pos' : 'Positive',
    'neg' : 'Negative'
}

for i in range(4):
    face_input = final_df['face_senti'][i]
    if face_input in face_numeric_to_emotion:
        face_input = face_numeric_to_emotion[face_input]
    else:
        face_input ='Neutrtal'

    voice_input = final_df['voice'][i]
    if voice_input in voice_numeric_to_emotion:
        voice_input = voice_numeric_to_emotion[voice_input]
    else:
        voice_input ='Neutrtal'


    text_input = final_df['text_sentiment'][i]
    if text_input in text_mapping:
        text_input = text_mapping[text_input]
    else:
        text_input ='Neutrtal'


    print(predict_emotion(face_input,voice_input, text_input))




Mixed Neutral-Angry-Neutral
Mixed Happy-Fearful-Neutral
Mixed Neutral-Angry-Neutral
Mixed Happy-Fearful-Neutral


In [15]:
def softmax_fusion(softmax1, softmax2, sentiment_scores, weights):
    # Normalize softmax outputs
    softmax1_normalized = softmax1 / np.sum(softmax1)
    softmax2_normalized = softmax2 / np.sum(softmax2)
    
    # Normalize sentiment scores
    sentiment_normalized = {}
    sentiment_sum = sum(sentiment_scores.values())
    for key, value in sentiment_scores.items():
        sentiment_normalized[key] = value / sentiment_sum
    
    # Combine the predictions
    fusion = softmax1_normalized * weights[0] + softmax2_normalized * weights[1]
    for key in sentiment_normalized:
        fusion += sentiment_normalized[key] * weights[2]
    
    return fusion